# Basic NanoVer networking

## Introduction

This notebook demonstrates how to connect to a standard NanoVer server and retrieve frame data with nothing more than a two technologies that are widely supported in many languages and environments: [WebSocket](https://websocket.org/resources/websocket-resources/#websocket-libraries-by-language) and [MessagePack](https://msgpack.org/#languages).

## Server setup

We setup a standard NanoVer server with one of the test files.

In [1]:
from nanover.omni import OmniRunner
from nanover.omni.openmm import OpenMMSimulation

simulation = OpenMMSimulation.from_xml_path("../openmm/openmm_files/17-ala.xml")
runner = OmniRunner.with_basic_server(simulation)

runner.load(0)
runner.print_basic_info()

Serving "ragzo: NanoVer iMD Server" (ws://localhost:62690), discoverable on all interfaces on port 54545
Available simulations:
0: "17-ala"


With the server running, we just need the connection address:

In [2]:
from nanover.websocket.client.app_client import get_websocket_address_from_app_server

ADDRESS = get_websocket_address_from_app_server(runner.app_server)
ADDRESS

'ws://localhost:62690'

## Connecting without NanoVer

It is fairly straight-forward to access a NanoVer server using just widely available libraries.

Using a standard WebSocket implemention, we can connect to the address from earlier and directly receive raw data:

In [3]:
from websockets.asyncio.client import connect

async with connect(ADDRESS) as websocket:
    print(await websocket.recv())
    print(await websocket.recv())

b'\x81\xa5frame\xde\x00\x18\xabframe.index\x00\xb9system.simulation.counter\x00\xb2particle.positions\xc5\x08\x1ck\x11Y\xbez\x835\xbd\x0fn\x1eA\xb5\xe0\xb3\xbd\x8bv\xa1;~u\x1dA\xd91\xba\xbb\xcf\'\xa9=`1\x1fA\xb4ll\xbd\x92\x94\xd0=\xb6\xf5 AW\xde\xf4\xbd\x9b\x1d\xa3=\xcbh\x1bA\xd5\xeb\x8a\xbeNq\xcf\xbd`\x8a\x1dA\x89\xd0H\xbe\xd63\xc4\xbd\x10\xcf\x1fA\x00G\x86\xbe>%\x17=\x86\xd4\x1eA\xb6V\n\xbdm\xe5\xae\xbds\x0f\x1dA\xad\xa0L\xbe\xaey\x88<c\x90\x1aA\xf3d\x1a\xbeR\xcf:>\x9e\x82\x1bAT@\x03\xbd\x18q\xac=\xcc|\x1aAOP\x00>\xfd\xd4\xd1=\xd7\xd9\x1eAt%a>8=<>?\x17 A\xc7O\xa4>\x01F\x82>\x99\xae\x1eA7@\xa5>\x8c\xf7h>k\xcb\x1cA\xa6\xd5\x90>c\x05\xd6=9\xd7!A\xcb\xc6(>\xd7z\x9e=\x0fw\x1dA\x00\x84,>@b\x87>G\xe9 A\x0b\xa6]>\xdfQo=\xdd\xe6"A\n\xd0\xae>\xd8\xf0\xd9<\x16\x0e!A\x19a\xb5>C\xd0/>\x1f\xcb"A\xbaN\xcf>\x98B\xb1>g\x8b\x1fA\x91\xb3\x06?\x0e\xd9\xc8>*\x83\x1eA$v\'?\x9c-\xa4>\xcb\x18\x1fA\xc6\xd5,?\xefb\x91>\xb3\xf4 A\xbb\xc4\t?h\xf5\n?\xe1\xcc\x1eA\xa8o\xc8>\xa2\xf3\xbc>\xae$!A4o\x01?H\x06\xbd>\xb

These raw bytes are in the MessagePack format, which we can decode easily:

In [4]:
import msgpack
import pprint

async with connect(ADDRESS) as websocket:
    data = await websocket.recv()
    message = msgpack.unpackb(data)
    pprint.pp(message)
    data = await websocket.recv()
    message = msgpack.unpackb(data)
    pprint.pp(message)

{'frame': {'frame.index': 0,
           'system.simulation.counter': 0,
           'particle.positions': b'8z\x8d\xbejeB=v1\x1eArG\x10\xbe\xd3G5='
                                 b'\xf4/\x1dA\x9b\t\xff\xbcI\xee\xbc=R\xad\x1eA'
                                 b' \xf4\x8c\xbd+*/>\xfa\n A\xb2\x9e\r\xbe'
                                 b'\xfa*\r>\xf31\x1bA\xb9\xaf\xb1\xbe5\xbb"='
                                 b'%\x17\x1dA\x9f\xd1\x92\xbeE\x9bW\xbc]t\x1fA'
                                 b'+(\x91\xbe9{\x10>\x14\xc4\x1eA0X\xf2\xbd'
                                 b'\xa1;e\xbd\x93\xc1\x1cA%\xddI\xbeI\x92\xb0='
                                 b'@\xe4\x19A\xfeC(\xbe|?s>\xe1\xaf\x1bA'
                                 b'\xb7e!\xbdb\x8e\x0c>.\x84\x1aA\xees\xc6='
                                 b'\x1c\xd6M=\xeet\x1eA\x8c\x1dY>g\x7f\xc5='
                                 b't\xba\x1fA\x04\xdb\xad>\xa9.\xe0=\x01g\x1eA'
                                 b'O\xf1\xb3>\x17\xae\x08=7\xed\x1cA\xcc\

Each message is a dictionary where each key is a message type and the value is the message of that type. We'll focus on the simulation and find the first message that has type "frame":

In [5]:
async with connect(ADDRESS) as websocket:
    async for data in websocket:
        message = msgpack.unpackb(data)
        if "frame" in message:
            FIRST_FRAME = message["frame"]
            break

pprint.pp(FIRST_FRAME)

{'frame.index': 0,
 'system.simulation.counter': 0,
 'particle.positions': b'c\xf5\x87\xbe\x7fI\x9f>Xn\x1dA\xdb\x98!\xbe\x82\xf5V>'
                       b'\xcbD\x1dA\xa8]R\xbd\xcccm>(\x0e\x1fA$\xb4\x91\xbd'
                       b'\x04\x16\xa6>eM A\t\xa9\xd1\xbdu\x0e^>\x8f\x11\x1bA'
                       b'\xf1\xa6\xaf\xbee\xab\x8a>\x1f\xa8\x1cA'
                       b'\xd2\x10\x8f\xbe\xf0\xca\xa7>N\xfc\x1eA\xa4\xdas\xbe'
                       b'\xc5:\xcd>X\xc4\x1cA\x03\xbeN\xbe\x9d\r\xe2=\xe8w\x1dA'
                       b'(\xb7/\xbe;\xa17>\xf6\xbb\x19A\xf4\xca\x9a\xbd'
                       b'\x98e\xa2>\xc5\x9f\x1aA{@\x93\xbbwD$>-\xf4\x1aA'
                       b'\x8c\xf0)=U\xa6\x08>/J\x1fA\xb7x.>\x8f]%>:i A'
                       b'\\\xeb\x8f>/\x8de>\x7f\x06\x1fA\x03\x89\x85>\x07\xddc>'
                       b'\x02\x0e\x1dA\x1d\x95]>wd\xd0<QL!Aw\xec-=\x1aRW='
                       b'\xd5E\x1eA\xe3\x07&>\xb7,f>B\xdd!A\x9c\xbe\x13>'
                       b'\x1d\x1dO\xbb

In general the frame messages provide continuous updates about the values of the simulated system as they change. The first frame message contains the entire initial state. The message is largely human-readable, but the positions are still raw bytes:

In [6]:
POSITION_DATA = FIRST_FRAME["particle.positions"]
POSITION_DATA

b'c\xf5\x87\xbe\x7fI\x9f>Xn\x1dA\xdb\x98!\xbe\x82\xf5V>\xcbD\x1dA\xa8]R\xbd\xcccm>(\x0e\x1fA$\xb4\x91\xbd\x04\x16\xa6>eM A\t\xa9\xd1\xbdu\x0e^>\x8f\x11\x1bA\xf1\xa6\xaf\xbee\xab\x8a>\x1f\xa8\x1cA\xd2\x10\x8f\xbe\xf0\xca\xa7>N\xfc\x1eA\xa4\xdas\xbe\xc5:\xcd>X\xc4\x1cA\x03\xbeN\xbe\x9d\r\xe2=\xe8w\x1dA(\xb7/\xbe;\xa17>\xf6\xbb\x19A\xf4\xca\x9a\xbd\x98e\xa2>\xc5\x9f\x1aA{@\x93\xbbwD$>-\xf4\x1aA\x8c\xf0)=U\xa6\x08>/J\x1fA\xb7x.>\x8f]%>:i A\\\xeb\x8f>/\x8de>\x7f\x06\x1fA\x03\x89\x85>\x07\xddc>\x02\x0e\x1dA\x1d\x95]>wd\xd0<QL!Aw\xec-=\x1aRW=\xd5E\x1eA\xe3\x07&>\xb7,f>B\xdd!A\x9c\xbe\x13>\x1d\x1dO\xbb\xbbf"A\x1a\xdce>\xdd\x1a\x0f\xbdR\xce\x1fA\xdf\xaf\x9f>\xbc\x0c\xd3<\xc86"Aq\xa2\xc5>\xe2]\x91>\xda\xdf\x1fA[_\x00?.9\xab>\xa9\x9d\x1eA\x19\x11#?\xdb$\xa3>g\xbe\x1fA\x83\xd6#?\xb2\x1a\x8c>_\x8e!A\xbfX\xec>\x02^\xf5>\xd7\x14\x1eA-\xad\xc9>\xf3\x81\x90>\xa5}!Aa\x11\x02?\x1a\xbb\x8e>\xfc\x1f\x1dA\xf3u\xb8>b\xb9\xf4>\xdbs\x1dA\xbd\x85\xe9>\xd8\x9b\n?&\x8a\x1fA-\xde\x06?i~\x04?\xea\xd6\x1cA\xd6\x0b>?

The positions are sent as an array of 32 bit floating point numbers, with three numbers per position, and 4 bytes per number, this 173 atom system should have 2076 bytes of positions data:

In [7]:
print(len(POSITION_DATA), len(POSITION_DATA) // 12)

2076 173


We can use numpy to easily transform these bytes into the array of position vectors (and in other languages and environments there are tools to do it just as easily):

In [8]:
import numpy as np

POSITIONS = np.frombuffer(POSITION_DATA, dtype=np.float32)
POSITIONS.reshape(-1, 3)

array([[-2.65544027e-01,  3.11107606e-01,  9.83943939e+00],
       [-1.57809660e-01,  2.09920913e-01,  9.82929516e+00],
       [-5.13588488e-02,  2.31826007e-01,  9.94095612e+00],
       [-7.11443722e-02,  3.24386716e-01,  1.00188951e+01],
       [-1.02373190e-01,  2.16852024e-01,  9.69178677e+00],
       [-3.43070537e-01,  2.70838886e-01,  9.79104519e+00],
       [-2.79425204e-01,  3.27720165e-01,  9.93659782e+00],
       [-2.38138735e-01,  4.00839001e-01,  9.79793549e+00],
       [-2.01896712e-01,  1.10377528e-01,  9.84177399e+00],
       [-1.71597123e-01,  1.79325983e-01,  9.60838890e+00],
       [-7.55824149e-02,  3.17181349e-01,  9.66400623e+00],
       [-4.49377066e-03,  1.60417423e-01,  9.68461323e+00],
       [ 4.14891690e-02,  1.33447006e-01,  9.95561123e+00],
       [ 1.70382366e-01,  1.61489710e-01,  1.00256901e+01],
       [ 2.81092525e-01,  2.24171385e-01,  9.93908596e+00],
       [ 2.60810941e-01,  2.22522840e-01,  9.81591988e+00],
       [ 2.16389135e-01,  2.54385304e-02